In [1]:
from datetime import datetime, timedelta

import pandas as pd
from demostore.driver_repo import transaction, transaction_stats_fv

from feast import FeatureStore, RepoConfig
import pytz

In [2]:
 fs = FeatureStore(repo_path="./demostore")

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fs.apply([transaction, transaction_stats_fv])

In [4]:
df=pd.read_csv("transaction_items.csv")

In [5]:
df

,CustomerID,price_sum,event_timestamp,created_timestamp
0,12346,-1,2010-06-30 11:22:00,2010-06-30 11:22:00
1,12349,60,2010-10-28 08:23:00,2010-10-28 08:23:00
2,12351,6,2010-11-29 15:23:00,2010-11-29 15:23:00
3,12355,35,2010-05-21 11:59:00,2010-05-21 11:59:00
4,12356,84,2010-10-11 09:42:00,2010-10-11 09:42:00
...,...,...,...,...
3109,18280,12,2010-11-10 15:51:00,2010-11-10 15:51:00
3110,18283,28,2010-11-22 15:01:00,2010-11-22 15:01:00
3111,18284,12,2010-10-04 11:33:00,2010-10-04 11:33:00
3112,18286,20,2010-08-20 11:57:00,2010-08-20 11:57:00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3114 entries, 0 to 3113
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CustomerID         3114 non-null   int64         
 1   price_sum          3114 non-null   int64         
 2   event_timestamp    3114 non-null   datetime64[ns]
 3   created_timestamp  3114 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 121.6 KB


In [7]:
df['event_timestamp']=pd.to_datetime(df['event_timestamp'],format='%Y-%m-%d %H:%M:%S')
df['created_timestamp']=pd.to_datetime(df['created_timestamp'],format='%Y-%m-%d %H:%M:%S')

In [9]:
training_df = fs.get_historical_features(
   feature_refs=[
       "transaction:price_sum",
       "transaction:total_count",
       "transaction:Country"
    ],
   entity_df=df
).to_df()

In [ ]:
print()

In [10]:
print(training_df)

               event_timestamp  CustomerID  price_sum  \
0    2009-12-01 10:36:00+00:00       16321          6   
1    2009-12-01 10:44:00+00:00       17592         -3   
2    2009-12-01 12:11:00+00:00       15299         -1   
3    2009-12-01 12:57:00+00:00       14654          6   
4    2009-12-01 14:52:00+00:00       17485         -1   
...                        ...         ...        ...   
3109 2010-12-09 16:53:00+00:00       15555         66   
3110 2010-12-09 18:58:00+00:00       14713         75   
3111 2010-12-09 19:23:00+00:00       14702         66   
3112 2010-12-09 19:28:00+00:00       13230         45   
3113 2010-12-09 20:01:00+00:00       17530         20   

             created_timestamp  transaction__price_sum  \
0    2009-12-01 10:36:00+00:00                    2.10   
1    2009-12-01 10:44:00+00:00                    4.25   
2    2009-12-01 12:11:00+00:00                    9.00   
3    2009-12-01 12:57:00+00:00                    0.42   
4    2009-12-01 14:52:00+

In [11]:
fs.list_feature_views()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[<FeatureView(name = transaction, entities = ['CustomerID'], features = [<feast.feature.Feature object at 0x7f7a3c3a88d0>, <feast.feature.Feature object at 0x7f7a3c3a8dd0>, <feast.feature.Feature object at 0x7f7a3c3a8c50>], tags = {'team': 'transaction'}, ttl = 7000 days, 0:00:00, online = True, input = <feast.data_source.BigQuerySource object at 0x7f7a3c3a8350>, materialization_intervals = [], created_timestamp = )>]

In [12]:
online_features = fs.get_online_features(
   feature_refs=[
       "transaction:price_sum",
       "transaction:total_count",
       "transaction:Country"
    ],
   entity_rows=[{"CustomerID": 12346}, {"CustomerID": 12419}],
).to_dict()

In [13]:
online_features

{'transaction__price_sum': [44.769999999999996, 28.0],
 'transaction__Country': ['United Kingdom', 'Spain'],
 'transaction__total_count': [3, 1],
 'CustomerID': [12346, 12419]}